# Backpropagation: The Heart of Deep Learning

Backpropagation is the algorithm that makes training neural networks possible. It efficiently computes gradients using the chain rule.

## What You'll Learn:
1. **The Chain Rule** - Mathematical foundation
2. **Computational Graphs** - Visual representation
3. **Forward and Backward Pass** - The two-phase algorithm
4. **Backprop from Scratch** - Complete NumPy implementation
5. **PyTorch Autograd** - Automatic differentiation
6. **Gradient Checking** - Verifying correctness

**Key Insight:** Backpropagation is just the chain rule applied systematically to compute derivatives.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

np.random.seed(42)
torch.manual_seed(42)

## 1. Review: The Chain Rule

The chain rule tells us how to differentiate composite functions.

### Single Variable:
If $y = f(g(x))$, then:

$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$

### Multiple Variables:
If $z = f(x, y)$ where $x = g(t)$ and $y = h(t)$, then:

$\frac{dz}{dt} = \frac{\partial z}{\partial x} \cdot \frac{dx}{dt} + \frac{\partial z}{\partial y} \cdot \frac{dy}{dt}$

In [ ]:
# Example: Chain rule in action
# Let's compute the derivative of f(x) = (2x + 1)^3

def f(x):
    """f(x) = (2x + 1)^3"""
    return (2*x + 1)**3

def df_dx_analytical(x):
    """
    Analytical derivative using chain rule:
    Let u = 2x + 1, so f = u^3
    df/dx = df/du * du/dx = 3u^2 * 2 = 6(2x + 1)^2
    """
    return 6 * (2*x + 1)**2

# Test at x = 2
x = 2.0
print(f"f({x}) = {f(x)}")
print(f"f'({x}) = {df_dx_analytical(x)}")

# Break down the chain rule step by step
u = 2*x + 1         # inner function
du_dx = 2           # derivative of inner function
df_du = 3 * u**2    # derivative of outer function
df_dx = df_du * du_dx  # chain rule!

print(f"\nChain rule breakdown:")
print(f"  u = 2x + 1 = {u}")
print(f"  du/dx = {du_dx}")
print(f"  df/du = 3u² = {df_du}")
print(f"  df/dx = df/du × du/dx = {df_dx}")

## 2. Computational Graphs

A **computational graph** represents mathematical operations as a directed graph:
- **Nodes**: Variables or operations
- **Edges**: Flow of values

### Example: $L = (w \cdot x + b)^2$

```
w, x, b (inputs)
   ↓
  mul: a = w * x
   ↓
  add: z = a + b
   ↓
 square: L = z^2
```

**Forward pass**: Compute values left to right  
**Backward pass**: Compute gradients right to left

In [ ]:
# Example: Simple computational graph
# L = (w * x + b)^2

# Inputs
w = 2.0
x = 3.0
b = 1.0

# Forward pass
a = w * x        # multiply
z = a + b        # add
L = z ** 2       # square

print("Forward Pass:")
print(f"  a = w * x = {w} * {x} = {a}")
print(f"  z = a + b = {a} + {b} = {z}")
print(f"  L = z^2 = {z}^2 = {L}")

# Backward pass (computing gradients)
# We want: dL/dw, dL/dx, dL/db

# Start from the end: dL/dL = 1
dL_dL = 1.0

# dL/dz = dL/dL * dL/dz = 1 * 2z = 2z
dL_dz = dL_dL * 2 * z

# dL/da = dL/dz * dz/da = dL/dz * 1
dL_da = dL_dz * 1

# dL/db = dL/dz * dz/db = dL/dz * 1
dL_db = dL_dz * 1

# dL/dw = dL/da * da/dw = dL/da * x
dL_dw = dL_da * x

# dL/dx = dL/da * da/dx = dL/da * w
dL_dx = dL_da * w

print("\nBackward Pass (Gradients):")
print(f"  dL/dz = 2z = {dL_dz}")
print(f"  dL/db = {dL_db}")
print(f"  dL/dw = dL/da * x = {dL_dw}")
print(f"  dL/dx = dL/da * w = {dL_dx}")

## 3. Backpropagation for a Single Neuron

Let's work through backprop for a single neuron step by step.

### Forward Pass:
1. $z = w^T x + b$ (linear combination)
2. $a = \sigma(z)$ (activation)
3. $L = \frac{1}{2}(y - a)^2$ (loss)

### Backward Pass (compute $\frac{\partial L}{\partial w}$ and $\frac{\partial L}{\partial b}$):

Using the chain rule:

$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot \frac{\partial z}{\partial w}$

Let's compute each piece:
1. $\frac{\partial L}{\partial a} = a - y$
2. $\frac{\partial a}{\partial z} = \sigma'(z) = \sigma(z)(1 - \sigma(z))$
3. $\frac{\partial z}{\partial w} = x$

Therefore:
$\frac{\partial L}{\partial w} = (a - y) \cdot \sigma'(z) \cdot x$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

# Example: Single neuron backprop
x = np.array([1.0, 2.0, 3.0])  # input
w = np.array([0.5, -0.2, 0.1]) # weights
b = 0.3                         # bias
y = 1.0                         # target

# Forward pass
z = np.dot(w, x) + b
a = sigmoid(z)
L = 0.5 * (y - a)**2

print("Forward Pass:")
print(f"  z = w·x + b = {z:.4f}")
print(f"  a = σ(z) = {a:.4f}")
print(f"  L = ½(y-a)² = {L:.4f}")

# Backward pass
dL_da = a - y                    # ∂L/∂a
da_dz = sigmoid_derivative(z)    # ∂a/∂z
dz_dw = x                        # ∂z/∂w
dz_db = 1.0                      # ∂z/∂b

# Chain rule
dL_dz = dL_da * da_dz            # ∂L/∂z
dL_dw = dL_dz * dz_dw            # ∂L/∂w
dL_db = dL_dz * dz_db            # ∂L/∂b

print("\nBackward Pass:")
print(f"  ∂L/∂a = a - y = {dL_da:.4f}")
print(f"  ∂a/∂z = σ'(z) = {da_dz:.4f}")
print(f"  ∂L/∂z = ∂L/∂a × ∂a/∂z = {dL_dz:.4f}")
print(f"  ∂L/∂w = {dL_dw}")
print(f"  ∂L/∂b = {dL_db:.4f}")

## 4. Backpropagation for Multi-Layer Networks

For a 2-layer network: $X \to Z^{[1]} \to A^{[1]} \to Z^{[2]} \to A^{[2]} \to L$

### Forward Pass:
1. $Z^{[1]} = W^{[1]}X + b^{[1]}$
2. $A^{[1]} = \sigma(Z^{[1]})$
3. $Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$
4. $A^{[2]} = \sigma(Z^{[2]})$
5. $L = \text{loss}(Y, A^{[2]})$

### Backward Pass:

Work backwards from the loss:

**Layer 2 (Output):**
1. $dZ^{[2]} = A^{[2]} - Y$ (for MSE loss)
2. $dW^{[2]} = \frac{1}{m} dZ^{[2]} (A^{[1]})^T$
3. $db^{[2]} = \frac{1}{m} \sum dZ^{[2]}$

**Layer 1 (Hidden):**
1. $dA^{[1]} = (W^{[2]})^T dZ^{[2]}$
2. $dZ^{[1]} = dA^{[1]} \odot \sigma'(Z^{[1]})$
3. $dW^{[1]} = \frac{1}{m} dZ^{[1]} X^T$
4. $db^{[1]} = \frac{1}{m} \sum dZ^{[1]}$

where $\odot$ is element-wise multiplication.

## 5. Complete Implementation from Scratch

In [ ]:
class NeuralNetworkWithBackprop:
    """
    Simple 2-layer neural network with backpropagation
    Architecture: input → hidden → output
    """
    
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Initialize weights with small random values
        self.W1 = np.random.randn(hidden_size, input_size) * 0.1
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size) * 0.1
        self.b2 = np.zeros((output_size, 1))
        
        self.learning_rate = learning_rate
    
    def relu(self, Z):
        return np.maximum(0, Z)
    
    def relu_derivative(self, Z):
        return (Z > 0).astype(float)
    
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-np.clip(Z, -500, 500)))
    
    def sigmoid_derivative(self, Z):
        s = self.sigmoid(Z)
        return s * (1 - s)
    
    def forward(self, X):
        """
        Forward propagation
        
        Args:
            X: input (input_size x m)
        
        Returns:
            cache: dict with intermediate values
        """
        # Layer 1
        Z1 = self.W1 @ X + self.b1
        A1 = self.relu(Z1)
        
        # Layer 2 (output)
        Z2 = self.W2 @ A1 + self.b2
        A2 = self.sigmoid(Z2)
        
        cache = {
            'X': X,
            'Z1': Z1,
            'A1': A1,
            'Z2': Z2,
            'A2': A2
        }
        
        return A2, cache
    
    def compute_loss(self, Y, A2):
        """Binary cross-entropy loss"""
        m = Y.shape[1]
        A2 = np.clip(A2, 1e-15, 1 - 1e-15)  # avoid log(0)
        loss = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
        return loss
    
    def backward(self, Y, cache):
        """
        Backpropagation
        
        Args:
            Y: true labels (output_size x m)
            cache: dict from forward pass
        
        Returns:
            gradients: dict with all gradients
        """
        m = Y.shape[1]
        X = cache['X']
        Z1 = cache['Z1']
        A1 = cache['A1']
        Z2 = cache['Z2']
        A2 = cache['A2']
        
        # Backward through layer 2
        dZ2 = A2 - Y  # derivative of binary cross-entropy + sigmoid
        dW2 = (1/m) * (dZ2 @ A1.T)
        db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
        
        # Backward through layer 1
        dA1 = self.W2.T @ dZ2
        dZ1 = dA1 * self.relu_derivative(Z1)
        dW1 = (1/m) * (dZ1 @ X.T)
        db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
        
        gradients = {
            'dW1': dW1,
            'db1': db1,
            'dW2': dW2,
            'db2': db2
        }
        
        return gradients
    
    def update_parameters(self, gradients):
        """Gradient descent update"""
        self.W1 -= self.learning_rate * gradients['dW1']
        self.b1 -= self.learning_rate * gradients['db1']
        self.W2 -= self.learning_rate * gradients['dW2']
        self.b2 -= self.learning_rate * gradients['db2']
    
    def train(self, X, Y, num_iterations=1000, print_every=100):
        """Complete training loop"""
        losses = []
        
        for i in range(num_iterations):
            # Forward pass
            A2, cache = self.forward(X)
            
            # Compute loss
            loss = self.compute_loss(Y, A2)
            losses.append(loss)
            
            # Backward pass
            gradients = self.backward(Y, cache)
            
            # Update parameters
            self.update_parameters(gradients)
            
            # Print progress
            if (i + 1) % print_every == 0:
                print(f"Iteration {i+1}/{num_iterations}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        """Make predictions"""
        A2, _ = self.forward(X)
        return (A2 > 0.5).astype(int)

print("Neural network with backpropagation implemented!")

### 5.1 Test on XOR Problem

In [ ]:
# XOR dataset
X_xor = np.array([[0, 0, 1, 1],
                  [0, 1, 0, 1]])
Y_xor = np.array([[0, 1, 1, 0]])

# Create and train network
nn = NeuralNetworkWithBackprop(input_size=2, hidden_size=4, output_size=1, learning_rate=0.5)
losses = nn.train(X_xor, Y_xor, num_iterations=2000, print_every=400)

# Plot loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss (XOR Problem)')
plt.grid(True, alpha=0.3)
plt.show()

# Test predictions
predictions = nn.predict(X_xor)
print("\nXOR Results:")
print("Input\t\tTarget\tPrediction")
print("-" * 40)
for i in range(4):
    print(f"[{X_xor[0,i]:.0f}, {X_xor[1,i]:.0f}]\t\t{Y_xor[0,i]}\t{predictions[0,i]}")

accuracy = np.mean(predictions == Y_xor) * 100
print(f"\nAccuracy: {accuracy:.1f}%")

## 6. Gradient Checking

**Gradient checking** verifies that our backprop implementation is correct by comparing analytical gradients with numerical gradients.

### Numerical Gradient:
$\frac{\partial L}{\partial \theta} \approx \frac{L(\theta + \epsilon) - L(\theta - \epsilon)}{2\epsilon}$

If the difference between analytical and numerical gradients is very small (< 1e-7), our implementation is correct!

In [ ]:
def gradient_check(nn, X, Y, epsilon=1e-7):
    """
    Verify backpropagation implementation using numerical gradients
    """
    # Get analytical gradients from backprop
    A2, cache = nn.forward(X)
    analytical_grads = nn.backward(Y, cache)
    
    # Check each parameter
    parameters = {'W1': nn.W1, 'b1': nn.b1, 'W2': nn.W2, 'b2': nn.b2}
    
    print("Gradient Checking:")
    print("-" * 60)
    
    for name, param in parameters.items():
        # Get analytical gradient
        analytical = analytical_grads[f'd{name}']
        
        # Compute numerical gradient for a few random elements
        numerical = np.zeros_like(param)
        
        # Sample a few random indices to check (checking all is slow)
        flat_param = param.ravel()
        flat_numerical = numerical.ravel()
        
        num_checks = min(10, len(flat_param))  # check up to 10 elements
        indices = np.random.choice(len(flat_param), num_checks, replace=False)
        
        for idx in indices:
            # Compute numerical gradient
            old_value = flat_param[idx]
            
            # L(theta + epsilon)
            flat_param[idx] = old_value + epsilon
            A2_plus, _ = nn.forward(X)
            loss_plus = nn.compute_loss(Y, A2_plus)
            
            # L(theta - epsilon)
            flat_param[idx] = old_value - epsilon
            A2_minus, _ = nn.forward(X)
            loss_minus = nn.compute_loss(Y, A2_minus)
            
            # Numerical gradient
            flat_numerical[idx] = (loss_plus - loss_minus) / (2 * epsilon)
            
            # Restore original value
            flat_param[idx] = old_value
        
        # Compute difference
        analytical_flat = analytical.ravel()
        difference = np.linalg.norm(analytical_flat[indices] - flat_numerical[indices]) / \
                     (np.linalg.norm(analytical_flat[indices]) + np.linalg.norm(flat_numerical[indices]) + 1e-8)
        
        status = "✓ PASS" if difference < 1e-5 else "✗ FAIL"
        print(f"{name}: difference = {difference:.2e} {status}")
    
    print("-" * 60)
    print("If all checks PASS, backpropagation is implemented correctly!")

# Run gradient check
np.random.seed(42)
nn_check = NeuralNetworkWithBackprop(2, 4, 1)
gradient_check(nn_check, X_xor, Y_xor)

## 7. PyTorch Autograd: Automatic Differentiation

PyTorch automatically computes gradients for us using **autograd**. No need to implement backprop manually!

### 7.1 Basic Autograd Example

In [ ]:
# Create tensors with gradient tracking
x = torch.tensor([2.0], requires_grad=True)
w = torch.tensor([3.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Forward pass: y = w*x + b
y = w * x + b
print(f"y = w*x + b = {y.item()}")

# Compute gradients automatically
y.backward()

# Access gradients
print(f"\nGradients:")
print(f"  dy/dx = {x.grad.item()} (should be w = {w.item()})")
print(f"  dy/dw = {w.grad.item()} (should be x = {x.item()})")
print(f"  dy/db = {b.grad.item()} (should be 1)")

### 7.2 Computational Graph in PyTorch

In [ ]:
# Example: L = (w * x + b)^2
x = torch.tensor([3.0], requires_grad=True)
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Forward
a = w * x
z = a + b
L = z ** 2

print(f"Forward: L = (w*x + b)^2 = ({w.item()}*{x.item()} + {b.item()})^2 = {L.item()}")

# Backward
L.backward()

print(f"\nGradients computed automatically:")
print(f"  dL/dw = {w.grad.item()}")
print(f"  dL/dx = {x.grad.item()}")
print(f"  dL/db = {b.grad.item()}")

# Verify manually
z_val = w.item() * x.item() + b.item()
dL_dw_manual = 2 * z_val * x.item()
print(f"\nManual calculation: dL/dw = 2*z*x = 2*{z_val}*{x.item()} = {dL_dw_manual}")

### 7.3 Training a Network with Autograd

In [ ]:
# XOR with PyTorch autograd
X = torch.tensor([[0., 0., 1., 1.],
                  [0., 1., 0., 1.]])
Y = torch.tensor([[0., 1., 1., 0.]])

# Initialize parameters
torch.manual_seed(42)
W1 = torch.randn(4, 2, requires_grad=True) * 0.1
b1 = torch.zeros(4, 1, requires_grad=True)
W2 = torch.randn(1, 4, requires_grad=True) * 0.1
b2 = torch.zeros(1, 1, requires_grad=True)

learning_rate = 0.5
losses = []

for i in range(2000):
    # Forward pass
    Z1 = W1 @ X + b1
    A1 = torch.relu(Z1)
    Z2 = W2 @ A1 + b2
    A2 = torch.sigmoid(Z2)
    
    # Loss
    loss = torch.mean((Y - A2)**2)
    losses.append(loss.item())
    
    # Backward pass (automatic!)
    loss.backward()
    
    # Update parameters
    with torch.no_grad():
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        
        # Zero gradients
        W1.grad.zero_()
        b1.grad.zero_()
        W2.grad.zero_()
        b2.grad.zero_()
    
    if (i + 1) % 400 == 0:
        print(f"Iteration {i+1}, Loss: {loss.item():.4f}")

# Plot
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training with PyTorch Autograd')
plt.grid(True, alpha=0.3)
plt.show()

# Test
with torch.no_grad():
    Z1 = W1 @ X + b1
    A1 = torch.relu(Z1)
    Z2 = W2 @ A1 + b2
    A2 = torch.sigmoid(Z2)
    predictions = (A2 > 0.5).float()

print("\nPredictions:")
print(predictions)
print("Targets:")
print(Y)

## 8. Understanding Gradient Flow

Let's visualize how gradients flow backward through the network.

In [ ]:
# Create a simple network and track gradients at each layer
class GradientTrackingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 1)
        self.activations = []
    
    def forward(self, x):
        self.activations = []
        
        x = self.fc1(x)
        x = torch.relu(x)
        self.activations.append(x)
        
        x = self.fc2(x)
        x = torch.relu(x)
        self.activations.append(x)
        
        x = self.fc3(x)
        x = torch.sigmoid(x)
        self.activations.append(x)
        
        return x

# Train briefly
model = GradientTrackingNet()
X_torch = torch.tensor([[0., 0., 1., 1.],
                        [0., 1., 0., 1.]], dtype=torch.float32).T
Y_torch = torch.tensor([[0., 1., 1., 0.]], dtype=torch.float32).T

# Single forward-backward pass
output = model(X_torch)
loss = nn.BCELoss()(output, Y_torch)
loss.backward()

# Examine gradients
print("Gradient magnitudes at each layer:")
print(f"Layer 1 (W): {model.fc1.weight.grad.abs().mean().item():.6f}")
print(f"Layer 2 (W): {model.fc2.weight.grad.abs().mean().item():.6f}")
print(f"Layer 3 (W): {model.fc3.weight.grad.abs().mean().item():.6f}")
print("\nNotice: Gradients often decrease in earlier layers (vanishing gradient problem)")

## Summary

### Key Takeaways:

1. **Backpropagation = Chain Rule**
   - Systematically apply chain rule to compute gradients
   - Work backward from loss to inputs
   - Reuse intermediate computations (efficient!)

2. **Two-Phase Algorithm**:
   - **Forward pass**: Compute activations, store values
   - **Backward pass**: Compute gradients using chain rule

3. **Implementation Details**:
   - Cache all intermediate values during forward pass
   - For each layer, compute: $dZ^{[l]}, dW^{[l]}, db^{[l]}$
   - Propagate gradient backwards: $dA^{[l-1]} = (W^{[l]})^T dZ^{[l]}$

4. **Gradient Checking**:
   - Verify backprop with numerical gradients
   - Should match to high precision (~1e-7)
   - Use only for debugging (slow!)

5. **PyTorch Autograd**:
   - Automatically builds computational graph
   - `.backward()` computes all gradients
   - No need to implement backprop manually!

### The Big Picture:

Backpropagation makes deep learning possible by efficiently computing gradients in deep networks. Without it, training would be prohibitively slow!

### Next Steps:
In the next notebook, we'll explore **optimization algorithms** - how to use these gradients to effectively train networks!

## Practice Exercises

In [ ]:
# Exercise 1: Implement backprop for a 3-layer network
# Extend NeuralNetworkWithBackprop to support 3 layers
# Your code here


In [ ]:
# Exercise 2: Compute gradients manually for this function
# L = (w1*x1 + w2*x2)^2 + (w1*x1 - w2*x2)^2
# Find dL/dw1 and dL/dw2
# Then verify with PyTorch autograd
# Your code here


In [ ]:
# Exercise 3: Implement gradient checking for all parameters
# Modify gradient_check to check ALL elements, not just a sample
# Your code here
